In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import time
import numpy as np
import math
'''
import sys
sys.path.append('Classes')
from arpy import *
'''
import arpa
from tqdm import tqdm

import torch
import torch.nn as nn



# prep cuda

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(torch.cuda.get_device_name(0))
print(torch.cuda.is_available())


GeForce GTX 1060 6GB
True


# Prep dict

In [32]:
#start = time.time()
file = '../../temp/12_14counts.txt'
#file = '../../temp/small_counts.txt'
first_read = open(file ,'r')

num_lines = sum(1 for line in open(file,'r'))
#print(num_lines)

ngram_dict = {}
for x in tqdm(first_read, total=num_lines, position=0, leave=True):
    line = x.split('\t')
    r = int(line[-1])
    ngram_dict[line[0]] = r



 52%|█████▏    | 8949863/17067352 [00:26<00:06, 1276168.87it/s]

  0%|          | 0/38836027 [00:00<?, ?it/s]

  0%|          | 122761/38836027 [00:00<00:31, 1227605.32it/s]

  1%|          | 232325/38836027 [00:00<00:32, 1184793.69it/s]

  1%|          | 349526/38836027 [00:00<00:33, 1141475.75it/s]

  1%|          | 465901/38836027 [00:00<00:33, 1148065.06it/s]

  2%|▏         | 583698/38836027 [00:00<00:33, 1156873.78it/s]

  2%|▏         | 699051/38836027 [00:00<00:36, 1044998.62it/s]

  2%|▏         | 817522/38836027 [00:00<00:35, 1083324.75it/s]

  2%|▏         | 934595/38836027 [00:00<00:34, 1108142.61it/s]

  3%|▎         | 1049556/38836027 [00:00<00:33, 1120264.81it/s]

  3%|▎         | 1159232/38836027 [00:01<00:33, 1113105.35it/s]

  3%|▎         | 1267845/38836027 [00:01<00:34, 1097671.06it/s]

  4%|▎         | 1379512/38836027 [00:01<00:33, 1103301.26it/s]

  4%|▍         | 1488561/38836027 [00:01<00:42, 881365.30it/s] 

  4%|▍         | 1599225/38836027 [00:01<00:39, 938

 33%|███▎      | 12844027/38836027 [00:12<00:23, 1092471.59it/s]

 33%|███▎      | 12964242/38836027 [00:12<00:23, 1123212.65it/s]

 34%|███▎      | 13081797/38836027 [00:13<00:22, 1138415.85it/s]

 34%|███▍      | 13200270/38836027 [00:13<00:22, 1151921.69it/s]

 34%|███▍      | 13316200/38836027 [00:13<00:22, 1120043.48it/s]

 35%|███▍      | 13428914/38836027 [00:13<00:23, 1090991.32it/s]

 35%|███▍      | 13539906/38836027 [00:13<00:23, 1096601.32it/s]

 35%|███▌      | 13658529/38836027 [00:13<00:22, 1122034.34it/s]

 35%|███▌      | 13777708/38836027 [00:13<00:21, 1142087.07it/s]

 36%|███▌      | 13893530/38836027 [00:13<00:21, 1146879.07it/s]

 36%|███▌      | 14011053/38836027 [00:13<00:21, 1155238.38it/s]

 36%|███▋      | 14128642/38836027 [00:13<00:21, 1161356.80it/s]

 37%|███▋      | 14246668/38836027 [00:14<00:21, 1166960.91it/s]

 37%|███▋      | 14363486/38836027 [00:14<00:21, 1163106.14it/s]

 37%|███▋      | 14479885/38836027 [00:14<00:20, 1161792.64it/s]

 38%|███▊ 

 66%|██████▌   | 25538667/38836027 [00:25<00:11, 1119209.20it/s]

 66%|██████▌   | 25650615/38836027 [00:25<00:11, 1116732.78it/s]

 66%|██████▋   | 25762309/38836027 [00:25<00:12, 1085342.22it/s]

 67%|██████▋   | 25871051/38836027 [00:25<00:12, 1050609.14it/s]

 67%|██████▋   | 25976499/38836027 [00:26<00:12, 1051047.12it/s]

 67%|██████▋   | 26081875/38836027 [00:26<00:12, 1049848.76it/s]

 67%|██████▋   | 26187050/38836027 [00:26<00:12, 1035932.11it/s]

 68%|██████▊   | 26293474/38836027 [00:26<00:12, 1044263.17it/s]

 68%|██████▊   | 26404683/38836027 [00:26<00:11, 1063722.23it/s]

 68%|██████▊   | 26511241/38836027 [00:26<00:11, 1048343.70it/s]

 69%|██████▊   | 26616256/38836027 [00:26<00:11, 1023548.56it/s]

 69%|██████▉   | 26718867/38836027 [00:26<00:11, 1014592.71it/s]

 69%|██████▉   | 26829756/38836027 [00:26<00:11, 1041152.05it/s]

 69%|██████▉   | 26939865/38836027 [00:26<00:11, 1058433.54it/s]

 70%|██████▉   | 27051962/38836027 [00:27<00:10, 1076447.44it/s]

 70%|█████

 98%|█████████▊| 37870426/38836027 [00:37<00:00, 1082623.27it/s]

 98%|█████████▊| 37978693/38836027 [00:37<00:00, 1077769.63it/s]

 98%|█████████▊| 38086479/38836027 [00:37<00:00, 1077487.25it/s]

 98%|█████████▊| 38194234/38836027 [00:37<00:00, 1071647.48it/s]

 99%|█████████▊| 38301411/38836027 [00:37<00:00, 1069448.07it/s]

 99%|█████████▉| 38409405/38836027 [00:37<00:00, 1072571.43it/s]

 99%|█████████▉| 38516975/38836027 [00:37<00:00, 1073507.63it/s]

 99%|█████████▉| 38624332/38836027 [00:37<00:00, 1066188.47it/s]

100%|█████████▉| 38730967/38836027 [00:37<00:00, 1066051.96it/s]

100%|██████████| 38836027/38836027 [00:37<00:00, 1023249.22it/s]


# Prep NN

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(4, 4)
        #self.fc2 = nn.Linear(4, 4)
        self.fc3 = nn.Linear(4, 3)
        self.fc4 = nn.Linear(3, 1)

        
    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))
        #x = torch.sigmoid(self.fc2(x))
        x = torch.sigmoid(self.fc3(x)) 
        x = torch.sigmoid(self.fc4(x)) 
        return x
    
net = Net().cuda()
net.load_state_dict(torch.load('NN saves/mse_4431_lr=0.01_m=0.9_without_r=1'))
net.eval()

Net(
  (fc1): Linear(in_features=4, out_features=4, bias=True)
  (fc3): Linear(in_features=4, out_features=3, bias=True)
  (fc4): Linear(in_features=3, out_features=1, bias=True)
)

# Load and Write ARPA

In [34]:
#start = time.time()
file = '../../temp/12-14o.arpa'
#file = '../../temp/small.arpa'
first_read = open(file ,'r')
new_file = open("../../temp/nn_12-14.arpa","w+")
#new_file = open('../../temp/nn_small.arpa' , 'w+')

num_lines = sum(1 for line in open(file,'r'))
current_ngram_len = 0

nn_input = torch.zeros(1, 4, dtype = torch.float, device = device)


for x in tqdm(first_read, total=num_lines, position=0, leave=True):
    if x == '\\end\\\n':
        current_ngram_len = -1
        new_file.write(x)
    elif x == '\n':
        new_file.write(x)
    elif current_ngram_len < 3:
        new_file.write(x)
    elif current_ngram_len == 3:
        #evaluate count
        line = x.split('\t')
        r = ngram_dict[line[1][:-1]]
        if r ==1 :
            print('oops')

        if r > 1 and r < 8: #only smooth values for r < 8
            ngram = line[1].split(' ')
            ngram[2] = ngram[2][:-1]

            ######setup nn input#######
            nn_input[0][0] = ngram_dict[ngram[0] + ' ' + ngram[1]]  #prefix count
            nn_input[0][1] = ngram_dict[line[1][:-1]]               #trigram count
            nn_input[0][2] = ngram_dict[ngram[1] + ' ' + ngram[2]]  #backoff bigram count
            nn_input[0][3] = ngram_dict[ngram[2]]                   #unigram count
            nn_input = 1/nn_input

            smoothed_prob  = net(nn_input)
            logbase = math.log(smoothed_prob, 10)

            #write
            new_file.write('{:.7f}\t{}\n'.format(logbase, line[1][:-1]))
        else:
            new_file.write(x)
            
    if x == '\\1-grams:\n':
        current_ngram_len = 1
    if x == '\\2-grams:\n':
        current_ngram_len = 2
    if x == '\\3-grams:\n':
        current_ngram_len = 3
        
new_file.close()


100%|██████████| 17067352/17067352 [46:57<00:00, 6058.25it/s]  
